In [23]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import joblib

In [24]:
# 1. 데이터 및 모델 로드
df = pd.read_csv("../data/cleaned_apt_trades_mapo_2024.csv")
model = joblib.load("../models/xgboost_price_model.pkl")

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2679 entries, 0 to 2678
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   apt_name       2679 non-null   object 
 1   district       2679 non-null   object 
 2   road_address   2679 non-null   object 
 3   price          2679 non-null   int64  
 4   area_m2        2679 non-null   float64
 5   floor          2679 non-null   int64  
 6   year_built     2679 non-null   int64  
 7   contract_date  2679 non-null   object 
 8   address        2679 non-null   object 
 9   lat            2679 non-null   float64
 10  lng            2679 non-null   float64
dtypes: float64(3), int64(3), object(5)
memory usage: 230.4+ KB


In [26]:
df[df['lat'].isna()]

,apt_name,district,road_address,price,area_m2,floor,year_built,contract_date,address,lat,lng


In [ ]:
# 공덕1삼성래미안
# 서울 마포구 마포대로 115-8,37.5459150093178,126.950622303108

# 예지안파크뷰
# 서울 마포구 방울내로1길 41-14,37.5594970166829,126.896596182273

In [27]:
# 2. 예측값 계산 및 가성비 지표 생성
features = ["area_m2", "floor", "year_built", "lat", "lng"]
df["predicted"] = model.predict(df[features])
df["value_ratio"] = df["price"] / df["predicted"]  # 1보다 작으면 예측보다 저렴

In [28]:
# 3. 지도 생성 (마포구 중심)
m = folium.Map(location=[37.55, 126.94], zoom_start=13)
marker_cluster = MarkerCluster().add_to(m)

In [29]:
# 4. 마커 추가
for _, row in df.iterrows():
    color = (
        "green"
        if row["value_ratio"] < 0.9
        else ("orange" if row["value_ratio"] <= 1.1 else "red")
    )
    tooltip = f"{row['apt_name']} ({row['contract_date']})\n\n실제: {row['price']:.0f} / 예측: {row['predicted']:.0f}"
    folium.CircleMarker(
        location=(row["lat"], row["lng"]),
        radius=6,
        color=color,
        fill=True,
        fill_opacity=0.7,
        tooltip=tooltip,
    ).add_to(marker_cluster)

In [ ]:
# 5. 저장
m.save("../outputs/mapo_price_map.html")
print("마포구 지도 저장 완료 : mapo_price_map.html")

마포구 지도 저장 완료 : mapo_price_map.html
